<a href="https://colab.research.google.com/github/upledge-general/finance-ai-community-public/blob/main/data-anonymization-masking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📌 Demo Notebook — Please Read First

Welcome! This is the exact notebook used in the live demo.

**How to use:**
1. Go to `Runtime` → `Run all`.
2. Wait until all cells finish (you’ll see ✅ messages).
3. Scroll and explore the outputs.

> 🔒 **Don't worry:** This original version is read-only.
> If you want to experiment, go to `File` → `Save a copy in Drive` and play in *your* copy.


# Data Anonymization / Masking / Redaction for LLM Use

Prepare **real-world data safely for LLM analysis** by automatically removing, hiding, or anonymizing sensitive information.

This notebook is designed for **non-technical users**.  
You don’t need to edit any code.

---

## How this notebook works

1. Click **Runtime → Run all**
2. Upload your file **or** paste your text
3. Choose:
   - Processing mode (**Redact / Mask / Anonymize**)
   - Policy level (**Light / Standard / Strict**)
4. Click **▶ Generate**
5. Download the sanitized outputs and audit files

That’s it.

---

## What happens behind the scenes

When you run this notebook, it will automatically:

- Prepare the required environment  
- Accept your uploaded file(s) or pasted text  
- Detect sensitive information such as:
  - emails, phone numbers, IDs
  - names, organisations, locations, addresses
  - money amounts, dates, tokens (Strict mode)
- Apply the selected transformation:
  - **Redact** → remove
  - **Mask** → partially hide
  - **Anonymize** → replace with safe placeholders
- Generate:
  - sanitized text / tables
  - a redaction log
  - a confidence report
  - an internal mapping dictionary

If something cannot be processed, the notebook will show a **clear, human-readable message** explaining why.

---

## First-time users (recommended)

For your first run, start with:
- **Pasted text**, or  
- A small **sample CSV**

This lets you quickly see the full anonymization flow before using real data.

You can re-run the notebook as many times as you like with different settings.

---

## Supported input formats

**Processed in this notebook**
- Text (`.txt`, `.md`)
- CSV (`.csv`)
- Excel (`.xlsx`)
- PDF *(text-based only)*
- JSON / JSONL (`.json`, `.jsonl`)
- Pasted raw text

**Not processed (advanced tier)**
- Scanned PDFs
- Images (`.png`, `.jpg`, etc.)
- Word / PowerPoint files (`.docx`, `.pptx`)

For these, the notebook will prompt you to **contact us** for advanced processing (OCR / layout parsing).

---

## About enable NER” option

When enabled, the notebook will also use AI models to detect:
- Person names
- Organisation names
- Locations
- Full addresses

This provides deeper protection but may take slightly longer to run.

You can turn it off for faster, rule-based processing.

---

## Tip

This notebook is safe to explore.  
Nothing is sent to external services.

If you want to try different settings, simply:
- change the options
- click **▶ Generate** again


#### Install & Imports

In [ ]:
# Install packages (once)
# uncomment this if you run for the first time/comment if you want to rerun multiple time
!pip install streamlit
!pip install -q pypdf
!pip install -q presidio-analyzer presidio-anonymizer spacy
!python -m spacy download en_core_web_sm

print("✅ Setup complete")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 19.0 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 7.0.0
    Uninstalling cachetools-7.0.0:
      Successfully uninstalled cachetools-7.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.6/330.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 109.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 8.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyopenssl 24.2.1 requires cryptography<44,>=41.0.5, but you have cryptography 46.0.5 which is incompatible

#### Web App

In [ ]:
%%writefile app.py
# app.py
# ============================================================
# Data Anonymization / Masking / Redaction for LLM Use
# Phase 1 (Regex) + Phase 2 (Presidio + spaCy NER)
#
# Supported (MVP):
#   .txt .md .csv .xlsx .pdf (text-based) .json .jsonl
#
# Advanced tier (skipped with message):
#   images (.png .jpg .jpeg .tiff .bmp .webp), .docx, .pptx, scanned PDFs
#
# Phase 2 NER:
#   - Runs only for Standard & Strict
#   - Uses Presidio Analyzer with spaCy en_core_web_sm
#   - Adds PERSON / ORG / LOCATION detections
#   - Adds ADDRESS detection "beyond basic" via:
#       (a) strong multi-pattern address span recognizer (regex-based, multi-lingual hints)
#       (b) upgrade rule: LOCATION -> ADDRESS when combined with postcode/street/unit cues
#
# Outputs:
#   - Anonymized text / Masked tables / Sanitized JSON(L)
#   - Redaction log (regex vs NER vs ADDRESS detector)
#   - Mapping dictionary (internal)
#   - Confidence report
#
# Run:
#   streamlit run app.py
# ============================================================

from __future__ import annotations

import io
import re
import json
import hashlib
from datetime import datetime
from typing import Any, Dict, List, Tuple, Optional

import pandas as pd
import streamlit as st

# Optional PDF text extraction
try:
    from pypdf import PdfReader  # type: ignore
    PYPDF_AVAILABLE = True
except Exception:
    PYPDF_AVAILABLE = False

# Optional NER (Phase 2)
try:
    from presidio_analyzer import AnalyzerEngine  # type: ignore
    from presidio_analyzer.nlp_engine import NlpEngineProvider  # type: ignore
    PRESIDIO_AVAILABLE = True
except Exception:
    PRESIDIO_AVAILABLE = False


# ---------------------------
# Product constants
# ---------------------------
SUPPORTED_SIMPLE = {"txt", "md", "csv", "xlsx", "pdf", "json", "jsonl"}
ADVANCED_TIER = {"png", "jpg", "jpeg", "tiff", "bmp", "webp", "docx", "pptx"}

CONTACT_URL = "https://upledge.io/contact-us"  # replace if needed


# ---------------------------
# Step 0: Policy definitions
# ---------------------------
# NOTE:
# - Standard / Strict include PERSON/ORG/LOCATION (via NER), and ADDRESS (custom detector).
POLICIES: Dict[str, List[str]] = {
    "Light": ["EMAIL", "PHONE", "ID"],
    "Standard": ["EMAIL", "PHONE", "ID", "IP", "URL", "PERSON", "ORG", "LOCATION", "ADDRESS"],
    "Strict": [
        "EMAIL", "PHONE", "ID", "IP", "URL", "PERSON", "ORG", "LOCATION", "ADDRESS",
        "CREDIT_CARD", "MONEY", "DATE", "TOKEN", "BEARER",
    ],
}


# ---------------------------
# Detection patterns (Phase 1)
# ---------------------------
RE_EMAIL = re.compile(r"\b[A-Za-z0-9._%+\-]+@[A-Za-z0-9.\-]+\.[A-Za-z]{2,}\b")
RE_PHONE = re.compile(
    r"(?<!\d)"
    r"(?:\+?\d{1,3}[\s\-]?)?"
    r"(?:\(?\d{2,4}\)?[\s\-]?)?"
    r"\d{3,4}[\s\-]?\d{3,4}"
    r"(?!\d)"
)
RE_IP = re.compile(r"\b(?:(?:25[0-5]|2[0-4]\d|1?\d?\d)\.){3}(?:25[0-5]|2[0-4]\d|1?\d?\d)\b")
RE_URL = re.compile(r"\bhttps?://[^\s]+\b", re.IGNORECASE)
RE_ID_GENERIC = re.compile(r"\b[A-Z]{1,3}\d{6,12}\b")  # generic IDs: ABC1234567

# Credit card (loose) + Luhn validation
RE_CC = re.compile(r"\b(?:\d[ -]*?){13,19}\b")

# Strict extras
RE_MONEY = re.compile(
    r"\b(?:RM|MYR|\$|USD|SGD|EUR|GBP)\s?\d{1,3}(?:,\d{3})*(?:\.\d+)?\b"
    r"|\b\d{1,3}(?:,\d{3})*(?:\.\d+)?\s?(?:RM|MYR|USD|SGD|EUR|GBP)\b"
)
RE_DATE = re.compile(r"\b(?:\d{1,2}[/-]\d{1,2}[/-]\d{2,4}|\d{4}[/-]\d{1,2}[/-]\d{1,2})\b")
RE_TOKEN = re.compile(r"\b(?:sk-|AIza|ghp_|xoxb-|xoxa-|AKIA)[A-Za-z0-9_\-]{10,}\b")
RE_BEARER = re.compile(r"\bBearer\s+[A-Za-z0-9\-\._~\+\/]+=*\b", re.IGNORECASE)


# ---------------------------
# Address detection (Phase 2 - beyond basic)
# ---------------------------
# The goal: detect real-world address spans reliably enough for redaction/masking,
# without claiming perfect postal normalization.
#
# Strategy:
# 1) Detect strong address lines using multiple patterns:
#    - unit/floor/block patterns (Unit/No/Level/Lot/Blk/Block)
#    - street indicators (Jalan/Jln/Lorong/Persiaran/Taman/Street/Road/Ave/etc.)
#    - postcode patterns (MY/SG/UK/US-like)
#    - state/city hints (common MY states + generic City/Province/State words)
# 2) Create spans and score them. Keep spans above a threshold.
# 3) Upgrade NER LOCATION spans to ADDRESS if they overlap or are adjacent to strong cues.

RE_POSTCODE_MY = re.compile(r"\b\d{5}\b")  # Malaysia
RE_POSTCODE_SG = re.compile(r"\b\d{6}\b")  # Singapore
RE_POSTCODE_UK = re.compile(r"\b[A-Z]{1,2}\d[A-Z\d]?\s?\d[A-Z]{2}\b", re.IGNORECASE)
RE_POSTCODE_US = re.compile(r"\b\d{5}(?:-\d{4})?\b")

RE_UNIT = re.compile(
    r"\b(?:no\.?|unit|suite|ste\.?|apt\.?|apartment|floor|lvl|level|lot|blok|blk|block|bldg|building)\b"
    r"(?:\s*[-#:]*\s*[A-Za-z0-9]+)?",
    re.IGNORECASE
)

RE_STREET = re.compile(
    r"\b(?:jalan|jln|lorong|persiaran|taman|lebuh|kg\.?|kampung|"
    r"street|st\.|road|rd\.|avenue|ave\.|lane|ln\.|drive|dr\.|boulevard|blvd\.|"
    r"place|pl\.|court|ct\.|crescent|cr\.|highway|hwy\.|"
    r"pusat|kompleks|seksyen|section|parcel|phase)\b",
    re.IGNORECASE
)

RE_LOCALITY_HINT = re.compile(
    r"\b(?:city|bandar|daerah|district|state|negeri|province|wilayah|"
    r"kedah|kelantan|terengganu|pahang|perlis|perak|selangor|johor|melaka|malacca|"
    r"penang|pulau pinang|sabah|sarawak|"
    r"kl|kuala lumpur|putrajaya|labuan)\b",
    re.IGNORECASE
)

# A "line-ish" address candidate: length-limited chunk that includes at least a street cue,
# plus optionally unit or postcode. We capture up to 140 chars to avoid grabbing whole paragraphs.
RE_ADDRESS_LINE = re.compile(
    r"(?P<addr>"
    r"(?:.{0,20}\b(?:no\.?|unit|lot|lvl|level|floor|blok|blk|block)\b.{0,40})?"
    r".{0,40}\b(?:jalan|jln|lorong|persiaran|taman|street|st\.|road|rd\.|avenue|ave\.|lane|ln\.|drive|dr\.)\b"
    r".{0,80}"
    r"(?:\b\d{5}\b|\b\d{6}\b|\b[A-Z]{1,2}\d[A-Z\d]?\s?\d[A-Z]{2}\b|\b\d{5}(?:-\d{4})?\b)?"
    r")",
    re.IGNORECASE | re.DOTALL
)


# ---------------------------
# Helpers
# ---------------------------
def get_ext(filename: str) -> str:
    if not filename or "." not in filename:
        return ""
    return filename.rsplit(".", 1)[-1].lower().strip()


def sha_token(value: str, prefix: str, salt: str) -> str:
    h = hashlib.sha256((salt + value).encode("utf-8")).hexdigest()[:8]
    return f"{prefix}_{h}"


def luhn_check(raw: str) -> bool:
    digits = [c for c in raw if c.isdigit()]
    if len(digits) < 13 or len(digits) > 19:
        return False
    s = 0
    parity = len(digits) % 2
    for i, ch in enumerate(digits):
        d = int(ch)
        if i % 2 == parity:
            d *= 2
            if d > 9:
                d -= 9
        s += d
    return s % 10 == 0


def mask_email(email: str) -> str:
    parts = email.split("@", 1)
    if len(parts) != 2:
        return "[MASKED:EMAIL]"
    user, dom = parts
    if len(user) <= 1:
        user_m = "*"
    elif len(user) == 2:
        user_m = user[0] + "*"
    else:
        user_m = user[0] + "***" + user[-1]
    return f"{user_m}@{dom}"


def mask_phone(phone: str) -> str:
    digits = "".join([c for c in phone if c.isdigit()])
    if len(digits) <= 4:
        return "[MASKED:PHONE]"
    return "****" + digits[-4:]


def advanced_tier_message(ext: str) -> str:
    return (
        f"This input (`.{ext}`) is in a **higher complexity tier**.\n\n"
        "It typically requires **advanced processing** such as OCR (for scanned docs/images) "
        "or layout-aware parsing (for DOCX/PPTX).\n\n"
        f"If you'd like to enable this, please contact us: {CONTACT_URL}"
    )


# ---------------------------
# Span utilities (safe replacement)
# ---------------------------
def drop_overlapping_spans(spans: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Keep non-overlapping spans. Prefer:
      1) earlier start
      2) longer length
      3) higher score (if present)
    """
    def sort_key(s: Dict[str, Any]):
        length = (s["end"] - s["start"])
        score = float(s.get("score", 0.0) or 0.0)
        return (s["start"], -length, -score)

    spans_sorted = sorted(spans, key=sort_key)
    kept: List[Dict[str, Any]] = []
    last_end = -1
    for s in spans_sorted:
        if s["start"] >= last_end:
            kept.append(s)
            last_end = s["end"]
    return kept


def apply_span_replacements(text: str, spans: List[Dict[str, Any]]) -> str:
    """
    Apply replacements from right-to-left so offsets remain valid.
    spans must have: start, end, replacement
    """
    out = text
    for s in sorted(spans, key=lambda x: (x["start"], x["end"]), reverse=True):
        out = out[:s["start"]] + s["replacement"] + out[s["end"]:]
    return out


# ---------------------------
# PDF extraction (text-based only)
# ---------------------------
def extract_pdf_text(pdf_bytes: bytes) -> Tuple[str, Dict[str, Any]]:
    if not PYPDF_AVAILABLE:
        return "", {"error": "pypdf is not available. Install pypdf to enable PDF extraction."}

    reader = PdfReader(io.BytesIO(pdf_bytes))
    pages = []
    empty_pages = 0

    for i, page in enumerate(reader.pages):
        try:
            t = page.extract_text() or ""
        except Exception:
            t = ""
        if not t.strip():
            empty_pages += 1
        pages.append(f"--- Page {i+1} ---\n{t}")

    text = "\n\n".join(pages).strip()
    likely_scanned = (len(reader.pages) > 0) and ((empty_pages / max(1, len(reader.pages))) > 0.6)

    meta = {"pages": len(reader.pages), "empty_pages": empty_pages, "likely_scanned": likely_scanned}
    return text, meta


# ---------------------------
# Presidio Analyzer (cached)
# ---------------------------
@st.cache_resource(show_spinner=False)
def get_presidio_analyzer() -> Optional["AnalyzerEngine"]:
    if not PRESIDIO_AVAILABLE:
        return None
    provider = NlpEngineProvider(
        nlp_configuration={
            "nlp_engine_name": "spacy",
            "models": [{"lang_code": "en", "model_name": "en_core_web_sm"}],
        }
    )
    nlp_engine = provider.create_engine()
    return AnalyzerEngine(nlp_engine=nlp_engine, supported_languages=["en"])


# ---------------------------
# Address detector (custom spans)
# ---------------------------
def detect_address_spans(text: str) -> List[Dict[str, Any]]:
    """
    Returns list of span dicts: start/end/original/entity_type/score/detector.
    Scoring is heuristic but stronger than "basic":
      - Street cue present -> +0.45
      - Unit cue present -> +0.20
      - Postcode present -> +0.25
      - Locality/state/city hint -> +0.10
      - Starts with number (common in addresses) -> +0.05
    Keep spans with score >= 0.55
    """
    spans: List[Dict[str, Any]] = []

    for m in RE_ADDRESS_LINE.finditer(text):
        addr = (m.group("addr") or "").strip()
        if not addr:
            continue

        start = m.start("addr")
        end = m.end("addr")
        snippet = addr

        score = 0.0
        if RE_STREET.search(snippet):
            score += 0.45
        if RE_UNIT.search(snippet):
            score += 0.20
        if RE_POSTCODE_MY.search(snippet) or RE_POSTCODE_SG.search(snippet) or RE_POSTCODE_UK.search(snippet) or RE_POSTCODE_US.search(snippet):
            score += 0.25
        if RE_LOCALITY_HINT.search(snippet):
            score += 0.10
        if re.match(r"^\s*\d{1,5}\b", snippet):
            score += 0.05

        # Avoid absurdly long captures (fallback safety)
        if (end - start) > 180:
            continue

        if score >= 0.55:
            spans.append({
                "entity_type": "ADDRESS",
                "start": start,
                "end": end,
                "original": text[start:end],
                "score": round(score, 3),
                "detector": "ADDRESS_REGEX",
            })

    return spans


def upgrade_location_to_address(
    text: str,
    ner_spans: List[Dict[str, Any]]
) -> List[Dict[str, Any]]:
    """
    If a LOCATION span overlaps OR is close to strong address cues,
    upgrade it to ADDRESS.
    """
    upgraded = []
    for s in ner_spans:
        if s["entity_type"] != "LOCATION":
            upgraded.append(s)
            continue

        # Examine a small window around the location
        w_start = max(0, s["start"] - 30)
        w_end = min(len(text), s["end"] + 50)
        window = text[w_start:w_end]

        evidence = 0
        if RE_STREET.search(window):
            evidence += 1
        if RE_UNIT.search(window):
            evidence += 1
        if RE_POSTCODE_MY.search(window) or RE_POSTCODE_SG.search(window) or RE_POSTCODE_UK.search(window) or RE_POSTCODE_US.search(window):
            evidence += 1
        if RE_LOCALITY_HINT.search(window):
            evidence += 1

        if evidence >= 2:
            s2 = dict(s)
            s2["entity_type"] = "ADDRESS"
            s2["detector"] = "NER_UPGRADED_TO_ADDRESS"
            # nudge score a bit for reporting
            s2["score"] = float(s2.get("score", 0.6) or 0.6) + 0.1
            upgraded.append(s2)
        else:
            upgraded.append(s)

    return upgraded


# ---------------------------
# Core: replace decision
# ---------------------------
def compute_replacement(
    raw: str,
    entity_type: str,
    mode: str,
    salt: str,
    mapping: Dict[str, str]
) -> str:
    if mode == "Redact":
        return f"[REDACTED:{entity_type}]"

    if mode == "Mask":
        if entity_type == "EMAIL":
            return mask_email(raw)
        if entity_type == "PHONE":
            return mask_phone(raw)
        if entity_type == "CREDIT_CARD":
            digits = "".join([c for c in raw if c.isdigit()])
            return "****" + (digits[-4:] if len(digits) >= 4 else "")
        # Generic mask for other entity types
        return f"[MASKED:{entity_type}]"

    # Anonymize
    key = f"{entity_type}::{raw}"
    if key not in mapping:
        mapping[key] = sha_token(raw, entity_type, salt)
    return mapping[key]


# ---------------------------
# Core: transform text (Phase 1 + Phase 2)
# ---------------------------
def transform_text(
    text: str,
    mode: str,          # "Redact" | "Mask" | "Anonymize"
    policy_name: str,   # "Light" | "Standard" | "Strict"
    salt: str,
    source: str,
    enable_ner: bool,
) -> Tuple[str, List[Dict[str, Any]], Dict[str, str], Dict[str, Any]]:
    enabled = set(POLICIES.get(policy_name, []))
    mapping: Dict[str, str] = {}
    log: List[Dict[str, Any]] = []

    # ---- Phase 1: Regex detections as span list (not re.sub), so we can merge with NER safely.
    # For regex, we build spans from the ORIGINAL text first.
    regex_detectors: List[Tuple[str, re.Pattern]] = []
    if "EMAIL" in enabled:
        regex_detectors.append(("EMAIL", RE_EMAIL))
    if "URL" in enabled:
        regex_detectors.append(("URL", RE_URL))
    if "IP" in enabled:
        regex_detectors.append(("IP", RE_IP))
    if "PHONE" in enabled:
        regex_detectors.append(("PHONE", RE_PHONE))
    if "CREDIT_CARD" in enabled:
        regex_detectors.append(("CREDIT_CARD", RE_CC))
    if "ID" in enabled:
        regex_detectors.append(("ID", RE_ID_GENERIC))
    if "TOKEN" in enabled:
        regex_detectors.append(("TOKEN", RE_TOKEN))
    if "BEARER" in enabled:
        regex_detectors.append(("BEARER", RE_BEARER))
    if "MONEY" in enabled:
        regex_detectors.append(("MONEY", RE_MONEY))
    if "DATE" in enabled:
        regex_detectors.append(("DATE", RE_DATE))

    spans: List[Dict[str, Any]] = []

    for etype, pattern in regex_detectors:
        for m in pattern.finditer(text):
            raw = m.group(0)

            # Reduce false positives for credit cards
            if etype == "CREDIT_CARD" and not luhn_check(raw):
                continue

            spans.append({
                "source": source,
                "entity_type": etype,
                "original": raw,
                "start": m.start(),
                "end": m.end(),
                "score": 0.99,  # regex detections are "high confidence" for what they match
                "detector": "REGEX",
            })

    # ---- Phase 2: ADDRESS detector (custom, beyond basic), only for Standard/Strict and only if enabled.
    if enable_ner and policy_name in {"Standard", "Strict"} and "ADDRESS" in enabled:
        addr_spans = detect_address_spans(text)
        # attach metadata
        for s in addr_spans:
            s.update({"source": source})
        spans.extend(addr_spans)

    # ---- Phase 2: NER (Presidio), only for Standard/Strict and only if enabled.
    if enable_ner and policy_name in {"Standard", "Strict"} and PRESIDIO_AVAILABLE:
        analyzer = get_presidio_analyzer()
        if analyzer is not None:
            entities = []
            if "PERSON" in enabled:
                entities.append("PERSON")
            if "ORG" in enabled:
                entities.append("ORGANIZATION")
            if "LOCATION" in enabled:
                entities.append("LOCATION")

            ner_results = analyzer.analyze(
                text=text,
                language="en",
                entities=entities if entities else None,
            )

            ner_spans: List[Dict[str, Any]] = []
            for r in ner_results:
                etype = r.entity_type
                if etype == "ORGANIZATION":
                    etype = "ORG"
                raw = text[r.start:r.end]

                ner_spans.append({
                    "source": source,
                    "entity_type": etype,
                    "original": raw,
                    "start": r.start,
                    "end": r.end,
                    "score": float(getattr(r, "score", 0.6) or 0.6),
                    "detector": "NER",
                })

            # Upgrade LOCATION -> ADDRESS when strong evidence exists around span
            if "ADDRESS" in enabled:
                ner_spans = upgrade_location_to_address(text, ner_spans)

            spans.extend(ner_spans)

    # If NER requested but Presidio missing, we still run Phase 1 + address regex. We'll report this later.

    # ---- Merge spans: remove overlaps
    spans = drop_overlapping_spans(spans)

    # ---- Compute replacements
    for s in spans:
        s["replacement"] = compute_replacement(
            raw=s["original"],
            entity_type=s["entity_type"],
            mode=mode,
            salt=salt,
            mapping=mapping,
        )
        log.append({
            "source": s["source"],
            "entity_type": s["entity_type"],
            "original": s["original"],
            "replacement": s["replacement"],
            "start": s["start"],
            "end": s["end"],
            "score": s.get("score", None),
            "detector": s.get("detector", None),
        })

    # ---- Apply replacements
    transformed = apply_span_replacements(text, spans)

    # ---- Confidence report
    counts: Dict[str, int] = {}
    counts_by_detector: Dict[str, int] = {}
    for row in log:
        counts[row["entity_type"]] = counts.get(row["entity_type"], 0) + 1
        det = row.get("detector") or "UNKNOWN"
        counts_by_detector[det] = counts_by_detector.get(det, 0) + 1

    limitations = [
        "Regex detection is deterministic but can miss contextual PII.",
    ]
    if policy_name in {"Standard", "Strict"}:
        if enable_ner:
            if not PRESIDIO_AVAILABLE:
                limitations.append("NER was enabled, but Presidio/spaCy is not installed in this environment.")
            else:
                limitations.append("NER (Presidio + spaCy) is enabled for PERSON/ORG/LOCATION; accuracy varies by text quality.")
        else:
            limitations.append("NER is disabled; Standard/Strict will not remove person/org names unless caught by regex.")
        limitations.append("ADDRESS detection uses heuristic multi-pattern matching; it may over/under-redact in edge cases.")

    confidence = {
        "source": source,
        "policy": policy_name,
        "mode": mode,
        "enable_ner": enable_ner,
        "total_replacements": len(log),
        "counts_by_entity": counts,
        "counts_by_detector": counts_by_detector,
        "limitations": limitations,
        "risk_flags": [
            "Review outputs for false positives/negatives when handling legal/compliance-grade documents.",
        ],
    }

    mapping_out = {f"{k}": v for k, v in mapping.items()}
    return transformed, log, mapping_out, confidence


# ---------------------------
# Tables: process dataframe
# ---------------------------
def process_dataframe(
    df: pd.DataFrame,
    mode: str,
    policy: str,
    salt: str,
    source: str,
    enable_ner: bool,
) -> Tuple[pd.DataFrame, List[Dict[str, Any]], Dict[str, str], Dict[str, Any]]:
    df_out = df.copy()
    logs_all: List[Dict[str, Any]] = []
    mapping_all: Dict[str, str] = {}
    counts_by_entity: Dict[str, int] = {}
    counts_by_detector: Dict[str, int] = {}

    for col in df_out.columns:
        # Keep numeric columns as-is (avoid breaking analytics)
        if pd.api.types.is_numeric_dtype(df_out[col]):
            continue

        new_vals = []
        for idx, val in enumerate(df_out[col].astype(str).tolist()):
            t, log, mapping, _ = transform_text(
                text=val,
                mode=mode,
                policy_name=policy,
                salt=salt,
                source=source,
                enable_ner=enable_ner,
            )
            new_vals.append(t)

            for r in log:
                r.update({"column": str(col), "row_index": idx})
                logs_all.append(r)
                counts_by_entity[r["entity_type"]] = counts_by_entity.get(r["entity_type"], 0) + 1
                det = r.get("detector") or "UNKNOWN"
                counts_by_detector[det] = counts_by_detector.get(det, 0) + 1

            mapping_all.update({f"{source}::{k}": v for k, v in mapping.items()})

        df_out[col] = new_vals

    confidence = {
        "source": source,
        "policy": policy,
        "mode": mode,
        "enable_ner": enable_ner,
        "total_replacements": sum(counts_by_entity.values()),
        "counts_by_entity": counts_by_entity,
        "counts_by_detector": counts_by_detector,
        "limitations": [
            "String-like columns are processed cell-by-cell; numeric columns are left unchanged.",
            "If IDs or account numbers are stored as numeric types, they may not be altered.",
        ],
    }
    return df_out, logs_all, mapping_all, confidence


# ---------------------------
# JSON/JSONL: sanitize string values recursively
# ---------------------------
def sanitize_json_obj(
    obj: Any,
    mode: str,
    policy: str,
    salt: str,
    source: str,
    enable_ner: bool,
    path: str = "$",
) -> Tuple[Any, List[Dict[str, Any]], Dict[str, str], Dict[str, Any]]:
    logs_all: List[Dict[str, Any]] = []
    mapping_all: Dict[str, str] = {}

    if isinstance(obj, str):
        t, log, mapping, conf = transform_text(
            text=obj,
            mode=mode,
            policy_name=policy,
            salt=salt,
            source=source,
            enable_ner=enable_ner,
        )
        for r in log:
            r.update({"json_path": path})
        logs_all.extend(log)
        mapping_all.update({f"{source}::{k}": v for k, v in mapping.items()})
        return t, logs_all, mapping_all, conf

    if isinstance(obj, list):
        out_list = []
        conf_any = None
        for i, item in enumerate(obj):
            v, l, m, c = sanitize_json_obj(item, mode, policy, salt, source, enable_ner, path=f"{path}[{i}]")
            out_list.append(v)
            logs_all.extend(l)
            mapping_all.update(m)
            conf_any = conf_any or c
        return out_list, logs_all, mapping_all, conf_any or {}

    if isinstance(obj, dict):
        out_dict = {}
        conf_any = None
        for k, v in obj.items():
            vv, l, m, c = sanitize_json_obj(v, mode, policy, salt, source, enable_ner, path=f"{path}.{k}")
            out_dict[k] = vv
            logs_all.extend(l)
            mapping_all.update(m)
            conf_any = conf_any or c
        return out_dict, logs_all, mapping_all, conf_any or {}

    return obj, logs_all, mapping_all, {}


# ---------------------------
# Streamlit UI
# ---------------------------
st.set_page_config(page_title="LLM Data Anonymization (Regex mode) + NER mode)", layout="wide")
st.title("Data Anonymization / Masking / Redaction for LLM Use")

st.markdown(
    """
### Please select input and policy mode to begin
**Choose input**
- Upload file(s) **OR** paste raw text

**Choose processing mode**
- **Redact**: remove sensitive data
- **Mask**: partially hide sensitive data
- **Anonymize**: replace with placeholders (best for LLM reasoning)

**Choose policy level**
- **Light**: email / phone / IDs (Regex mode)
- **Standard**: Light + PERSON/ORG/LOCATION + ADDRESS (NER mode enable)
- **Strict**: Standard + amounts + dates + internal IDs + tokens (NER mode enable)

**Supported in this app**
- `.txt .md .csv .xlsx .pdf (text-based) .json .jsonl`

**Advanced tier (not processed)**
- images / scanned PDFs / `.docx` / `.pptx` → requires OCR or layout-aware parsing
If uploaded, the app will show a message to contact us.
"""
)

with st.sidebar:
    st.header("Settings")
    mode = st.selectbox("Mode", ["Anonymize", "Mask", "Redact"])
    policy = st.selectbox("Policy Level", ["Light", "Standard", "Strict"])
    salt = st.text_input("Session Salt (stable anonymized tokens)", value="upledge-demo-salt")

    st.markdown("---")
    enable_ner = st.checkbox("Enable this button (PERSON/ORG/LOCATION + ADDRESS) if you want “aggressive” masking", value=True)

    if enable_ner and policy in {"Standard", "Strict"} and not PRESIDIO_AVAILABLE:
        st.warning("NER is enabled but Presidio/spaCy isn't available. Install presidio-analyzer + spacy + en_core_web_sm.")

    st.markdown("---")
    if not PYPDF_AVAILABLE:
        st.warning("PDF extraction is disabled (pypdf not available). Install `pypdf` to enable text-based PDF support.")

    st.link_button("Contact us (OCR / advanced tier)", CONTACT_URL)

left, right = st.columns([1, 1])

with left:
    st.subheader("Option A: Upload file(s)")
    uploaded = st.file_uploader(
        "Upload (multi-file supported)",
        type=sorted(list(SUPPORTED_SIMPLE | ADVANCED_TIER)),
        accept_multiple_files=True,
    )

with right:
    st.subheader("Option B: Paste raw text")
    pasted = st.text_area(
        "Paste text here (optional)",
        height=240,
        placeholder="Paste transcript, email, notes, logs, etc.",
    )

run = st.button("Generate", type="primary")


# ---------------------------
# Execute run
# ---------------------------
if run:
    started_at = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    st.success(f"Run started at {started_at}")

    outputs: List[Tuple[str, Any]] = []
    logs_all: List[Dict[str, Any]] = []
    mapping_all: Dict[str, str] = {}
    confidence_reports: List[Dict[str, Any]] = []
    skipped: List[Dict[str, str]] = []

    # 1) Pasted text
    if pasted and pasted.strip():
        out_text, log, mapping, conf = transform_text(
            text=pasted.strip(),
            mode=mode,
            policy_name=policy,
            salt=salt,
            source="pasted_text",
            enable_ner=enable_ner,
        )
        outputs.append(("pasted_text.sanitized.txt", out_text))
        logs_all.extend(log)
        mapping_all.update({f"pasted_text::{k}": v for k, v in mapping.items()})
        confidence_reports.append(conf)

    # 2) Uploaded files
    if uploaded:
        for f in uploaded:
            filename = f.name
            ext = get_ext(filename)
            raw = f.read()

            # Advanced tier gate
            if ext in ADVANCED_TIER:
                skipped.append({"file": filename, "reason": f"Advanced tier (.{ext})"})
                st.warning(f"⚠️ Skipped: {filename}")
                st.info(advanced_tier_message(ext))
                continue

            # Unsupported gate
            if ext not in SUPPORTED_SIMPLE:
                skipped.append({"file": filename, "reason": f"Unsupported type (.{ext or 'unknown'})"})
                st.warning(f"⚠️ Skipped: {filename}")
                st.info(advanced_tier_message(ext or "unknown"))
                continue

            # Supported handlers
            if ext in {"txt", "md"}:
                text = raw.decode("utf-8", errors="replace")
                out_text, log, mapping, conf = transform_text(text, mode, policy, salt, source=filename, enable_ner=enable_ner)
                outputs.append((f"{filename}.sanitized.txt", out_text))
                logs_all.extend(log)
                mapping_all.update({f"{filename}::{k}": v for k, v in mapping.items()})
                confidence_reports.append(conf)

            elif ext == "pdf":
                text, meta = extract_pdf_text(raw)

                if not text.strip() or meta.get("likely_scanned"):
                    skipped.append({"file": filename, "reason": "Likely scanned PDF (OCR required)"})
                    st.warning(f"⚠️ Skipped: {filename}")
                    st.info(advanced_tier_message("scanned pdf"))
                    continue

                out_text, log, mapping, conf = transform_text(text, mode, policy, salt, source=filename, enable_ner=enable_ner)
                conf = {**conf, "pdf_meta": meta}
                outputs.append((f"{filename}.sanitized.txt", out_text))
                logs_all.extend(log)
                mapping_all.update({f"{filename}::{k}": v for k, v in mapping.items()})
                confidence_reports.append(conf)

            elif ext == "csv":
                df = pd.read_csv(io.BytesIO(raw))
                df_out, log, mapping, conf = process_dataframe(df, mode, policy, salt, source=filename, enable_ner=enable_ner)
                outputs.append((f"{filename}.sanitized.csv", df_out))
                logs_all.extend(log)
                mapping_all.update(mapping)
                confidence_reports.append(conf)

            elif ext == "xlsx":
                df = pd.read_excel(io.BytesIO(raw))
                df_out, log, mapping, conf = process_dataframe(df, mode, policy, salt, source=filename, enable_ner=enable_ner)
                outputs.append((f"{filename}.sanitized.xlsx", df_out))
                logs_all.extend(log)
                mapping_all.update(mapping)
                confidence_reports.append(conf)

            elif ext == "json":
                data = json.loads(raw.decode("utf-8", errors="replace"))
                sanitized, log, mapping, conf = sanitize_json_obj(data, mode, policy, salt, source=filename, enable_ner=enable_ner)
                outputs.append((f"{filename}.sanitized.json", json.dumps(sanitized, ensure_ascii=False, indent=2)))
                logs_all.extend(log)
                mapping_all.update(mapping)
                confidence_reports.append(conf or {"source": filename})

            elif ext == "jsonl":
                lines = raw.decode("utf-8", errors="replace").splitlines()
                out_lines: List[str] = []
                log_acc: List[Dict[str, Any]] = []
                map_acc: Dict[str, str] = {}
                conf_any = None

                for i, line in enumerate(lines):
                    if not line.strip():
                        continue
                    obj = json.loads(line)
                    sanitized, log, mapping, conf = sanitize_json_obj(
                        obj, mode, policy, salt, source=filename, enable_ner=enable_ner, path=f"$[line={i}]"
                    )
                    out_lines.append(json.dumps(sanitized, ensure_ascii=False))
                    log_acc.extend(log)
                    map_acc.update(mapping)
                    conf_any = conf_any or conf

                outputs.append((f"{filename}.sanitized.jsonl", "\n".join(out_lines)))
                logs_all.extend(log_acc)
                mapping_all.update(map_acc)
                confidence_reports.append(conf_any or {"source": filename})

    # ---------------------------
    # Display outputs
    # ---------------------------
    st.markdown("---")
    st.subheader("Outputs")

    if not outputs:
        st.warning("No supported inputs were processed. Upload a supported file or paste text.")
    else:
        for fname, payload in outputs:
            st.write(f"**{fname}**")
            if isinstance(payload, pd.DataFrame):
                st.dataframe(payload, use_container_width=True)
                csv_bytes = payload.to_csv(index=False).encode("utf-8")
                st.download_button(
                    "Download table as CSV",
                    data=csv_bytes,
                    file_name=fname.replace(".xlsx", ".csv"),
                    mime="text/csv",
                    key=f"dl_{fname}",
                )
            else:
                preview = str(payload)
                st.text_area("Preview", value=preview[:20000], height=220, key=f"prev_{fname}")
                st.download_button(
                    "Download file",
                    data=preview.encode("utf-8"),
                    file_name=fname,
                    mime="text/plain",
                    key=f"dl_{fname}",
                )
            st.markdown("---")

    # ---------------------------
    # Audit artifacts
    # ---------------------------
    st.subheader("Audit Artifacts")

    if skipped:
        st.write("**Skipped inputs**")
        st.dataframe(pd.DataFrame(skipped), use_container_width=True)

    if logs_all:
        df_log = pd.DataFrame(logs_all)
        st.write("**Redaction / Masking Log**")
        st.dataframe(df_log.head(800), use_container_width=True)
        st.download_button(
            "Download redaction_log.csv",
            data=df_log.to_csv(index=False).encode("utf-8"),
            file_name="redaction_log.csv",
            mime="text/csv",
        )
    else:
        st.info("No log entries (either nothing detected or no supported inputs were processed).")

    # Confidence report
    conf_payload = {
        "generated_at": started_at,
        "global_settings": {"mode": mode, "policy": policy, "enable_ner": enable_ner},
        "reports": confidence_reports,
    }
    st.write("**Confidence Report**")
    st.json(conf_payload)
    st.download_button(
        "Download confidence_report.json",
        data=json.dumps(conf_payload, ensure_ascii=False, indent=2).encode("utf-8"),
        file_name="confidence_report.json",
        mime="application/json",
    )

    # Mapping dictionary (internal)
    st.write("**Mapping Dictionary (internal)**")
    st.caption("Keep this internal. Sharing it externally can re-identify the data.")
    preview_map = dict(list(mapping_all.items())[:120])
    st.json(preview_map)
    st.download_button(
        "Download mapping_dictionary.json",
        data=json.dumps(mapping_all, ensure_ascii=False, indent=2).encode("utf-8"),
        file_name="mapping_dictionary.json",
        mime="application/json",
    )

    st.markdown("---")
    st.caption("Standard/Strict Mode: Enable Presidio+spaCy NER (PERSON/ORG/LOCATION) + stronger ADDRESS detection. Advanced tier still requires OCR/layout parsing. Please contact us for advanced use case")


Writing app.py


In [ ]:
!pkill -f streamlit || true
!streamlit run app.py --server.port 8501 --server.address 0.0.0.0 \
  --server.headless true --server.enableCORS false --server.enableXsrfProtection false \
  &>/content/streamlit.log &


^C


In [ ]:
!curl -I http://127.0.0.1:8501 | head


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (7) Failed to connect to 127.0.0.1 port 8501 after 0 ms: Connection refused


In [ ]:
from google.colab import output
output.serve_kernel_port_as_window(8501)
print("Proceess done ✅")
print("Please use Chrome browser to use the web app link above")

Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

Proceess done ✅
Please use Chrome browser to use the web app link above


<div style="text-align:center; margin-top:40px;">

  <h2>🚀 Want to Go Further?</h2>

  <p style="font-size:16px; max-width:600px; margin:0 auto 24px;">
    This notebook is just a starting point.
    If you’d like to explore this with your own data or in a real-world setup, we’re happy to help.
  </p>

  <a href="https://upledge.io/contact-us"
     target="_blank"
     rel="noopener noreferrer"
     style="
       display:inline-block;
       padding:14px 28px;
       background-color:#2563EB;
       color:#ffffff;
       text-decoration:none;
       border-radius:8px;
       font-size:16px;
       font-weight:600;
       box-shadow:0 4px 10px rgba(0,0,0,0.15);
     ">
     Contact Us
  </a>

</div>
